## Description
Generates PSFM for alignment cores from each cluster set files provided in the `paths`. Calculate then the KL divergence between each possible combination of cluster vs cluster (one to one and one to many) divergence and plots.
For each cluster set, the distribution of KLD values can be visualized and compared between different clustering parameters. To plot several kld distribution and compare them, a pickle file should be created beforehand storing each KLD sets.

In [1]:
# set the variables for the experiment:
path_to_gibbs_cluster_output = "/home/daqop/mountpoint_snellius/softwares/gibbscluster-2.0/run/all_hla_peptides_anchor_representation_1_15_clusters_no_trash_1075335/cores/gibbs.*of*.core"
experiment_title = "pep_representation"
test_filename = "all_kld_tests_lambda_j.pkl" # file where all sets of KLDs values are saved to be plotted together
exp = "one2many" #one2many or one2one
title_for_plot = "test" # title of the plot for the saved file pkl file

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
import re
import plotly.graph_objects as go
import pickle

In [3]:
paths = glob.glob(path_to_gibbs_cluster_output)
alphabet = list("ARNDCQEGHILKMFPSTWYV") # universe of possible AA
pseudocount = 10 # to prevent 0 in the matrice
p_matrices = [] # PSFM matrices are stacked here
p_matrice_names = [] # IDs for each PSFM
for path in paths:
    core_f = open(path, "r")
    f_name = path.split("/")[-1].replace(".core", "")
    # using regexp extract the cluster set and cluster name from the filename
    cluster = int(re.search("(?<=gibbs.)[0-9]*", f_name).group()) 
    cluster_set = int(re.search("(?<=of)[0-9]*", f_name).group())
    p_matrice_names.append({
        "cluster": cluster,
        "cluster_set": cluster_set
    })
    peptides = [] # 9-mer cores of the alignment file
    for l in core_f:
        peptides.append(list(l.replace("\n","")))
    peptides = np.array(peptides)
    psfm_matrice_for_one_cluster = np.stack([
        (np.count_nonzero(peptides == alphabet[i], axis=0)+pseudocount) / (peptides.shape[0]+pseudocount*len(alphabet)) 
        for i in range(len(alphabet))
    ])
    p_matrices.append(psfm_matrice_for_one_cluster)
p_matrices = np.stack(p_matrices)

In [4]:
# calculate one-to-one
all_klds = [] # all KL divergences for each cluster vs cluster comparison
all_kld_indices = [] # ID of each KLD value
for i in range(1,16):
    for j in range(1, i+1):
        # Calculates the divergence between the matrice of cluster p and compare to matrice of cluster q.
        # For n the number of cluster in each cluster set, the number of KLD values is n^2
        p_matrice = [p_matrices[p_matrice_names.index(name)] for name in p_matrice_names if name["cluster"] == j and name["cluster_set"] == i][0]
        for k in range(1, i+1):
            q_matrice = [p_matrices[p_matrice_names.index(name)] for name in p_matrice_names if name["cluster_set"] == i and name["cluster"] == k][0]
            kld_pq = np.sum(p_matrice*np.log(p_matrice/q_matrice), axis=0)
            all_kld_indices.append(
                [j,k,i] # p,q and cluster_set define each KLD value
            )
            all_klds.append(kld_pq)
all_klds = np.stack(all_klds)
all_kld_indices = np.array(all_kld_indices)

In [5]:
# now that we have every KLD values for each cluster vs cluster combination in each cluster set, each
# values are saved in a pickle file to be ploted later with different sets of KLDs
all_klds_condition = {
    "title": experiment_title, # this title will be used as legend in the plot
    "klds": [] # KLD values go in here
}

one2many_mean_klds = []
for i in range(1,16):
    if exp == "one2one":
        cluster_set_indices = [ind for ind in range(len(all_kld_indices)) if all_kld_indices[ind][2] == i]
        # actually the KLD distance is calculated for each position in the PSFM. Thus, the KLD of one cluster
        # to the other is defined by a vector of length 9. Each cluster-to-cluster divergence is defined by
        # the mean value over the vector:
        klds = all_klds[cluster_set_indices].mean(axis=1, dtype=np.float64)

        # # if i == 10: # this line is used to explore a specific cluster set with sorted distances
        # #     sorted_indices = np.argsort(klds)+j
        # #     indices = all_kld_indices[sorted_indices]
        # #     print(indices[-10:]) # get the 10 most distance combination of cluster vs cluster.
        all_klds_condition["klds"].append(klds)
    if exp == "one2many":
        for j in range(1, i+1):
            cluster_set_indices = [ind for ind in range(len(all_kld_indices))
                if all_kld_indices[ind][2] == i and all_kld_indices[ind][0] == j
            ]
            klds = all_klds[cluster_set_indices].mean(axis=1, dtype=np.float64)
            one2many_mean_klds.append({
                "cluster_set": i,
                "cluster": j,
                "mean_kld_for_cluster": klds.mean()
            })
if exp == "one2many":
    kld_df = pd.DataFrame(one2many_mean_klds)
    # kld_group = kld_df.groupby(["cluster_set"])["klds"]
    kld_df['cluster_max_distance'] = kld_df.groupby(['cluster_set'])['mean_kld_for_cluster'].transform(max)
    print(kld_df.loc[kld_df["cluster_max_distance"] == kld_df["mean_kld_for_cluster"]])

# all_kld_tests_arr = pickle.load(open(test_filename, "rb"))
# all_kld_tests_arr.append(all_klds_condition)
# pickle.dump(all_kld_tests_arr, open(test_filename, "wb"))

     cluster_set  cluster  mean_kld_for_cluster  cluster_max_distance
0              1        1              0.000000              0.000000
2              2        2              0.339755              0.339755
5              3        3              0.367206              0.367206
9              4        4              0.286381              0.286381
13             5        4              0.314231              0.314231
17             6        3              0.382641              0.382641
25             7        5              0.398325              0.398325
32             8        5              0.406525              0.406525
38             9        3              0.413006              0.413006
54            10       10              0.450386              0.450386
62            11        8              0.453877              0.453877
67            12        2              0.478884              0.478884
82            13        5              0.471028              0.471028
102           14    

In [6]:
# plot KLDs from a pkl file
fig = go.Figure()
all_kld_tests = pickle.load(open("all_kld_tests_lambda_j.pkl", "rb"))
x = []
for i in range(len(all_kld_tests)):
    for j in range(1,16):
        x.extend([j]*(j*j))
for i in range(len(all_kld_tests)):
    klds = []
    cond = all_kld_tests[i]
    for k in cond["klds"]:
        klds.extend(k.tolist())
    fig.add_trace(
        go.Box(
            x = x,
            y = klds,
            name = cond["title"]
        )
    )
fig.update_layout(
    title="The distribution of KL divergence values between clusters for each cluster set",
    xaxis_title="Cluster set",
    yaxis_title="Cluster-cluster divergence in one cluster set",
    boxmode="group",
    autosize=False,
    width = 1600,
    height = 800
)
fig.show()
fig.write_image(f"../../reports/figures/gibbs_clusters/{title_for_plot}.svg")